In [58]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from PIL import Image
import os
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# unzip (ZIP)
# !unzip "/content/drive/MyDrive/data/archive.zip" -d "/content/drive/MyDrive/data/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
'''
ZIP_PATH = "/content/drive/MyDrive/data/archive.zip"
EXTRACT_TO = "/content/drive/MyDrive/data_extracted"# clean new folder
!mkdir -p "$EXTRACT_T
!unzip -q "$ZIP_PATH" -d "$EXTRACT_TO"
print("Extraction complete")
'''

'\nZIP_PATH = "/content/drive/MyDrive/data/archive.zip"\nEXTRACT_TO = "/content/drive/MyDrive/data_extracted"# clean new folder\n!mkdir -p "$EXTRACT_T\n!unzip -q "$ZIP_PATH" -d "$EXTRACT_TO"\nprint("Extraction complete")\n'

In [ ]:
import os

print("Top-level extracted folders/files:")
print(os.listdir(EXTRACT_TO))

# Look one level deeper if needed
for item in os.listdir(EXTRACT_TO):
    path = os.path.join(EXTRACT_TO, item)
    if os.path.isdir(path):
        print(f"\nContents of {item}:")
        print(os.listdir(path)[:10])  # show only first


Top-level extracted folders/files:
['images', 'labels']

Contents of images:
['20160928-140314-0.jpg', '20160928-140337-0.jpg', '20160928-140731-0.jpg', '20160928-140747-0.jpg', '20160928-141107-0.jpg', '20160928-141135-0.jpg', '20160928-141355-0.jpg', '20160928-141421-0.jpg', '20160928-141437-0.jpg', '20160928-142056-0.jpg']

Contents of labels:
['labels.csv', 'test_subset0.csv', 'test_subset1.csv', 'test_subset2.csv', 'test_subset3.csv', 'test_subset4.csv', 'train_subset0.csv', 'train_subset1.csv', 'train_subset2.csv', 'train_subset3.csv']


In [ ]:

main = "/content/drive/MyDrive/data_extracted"

# Quick check: list the labels folder
labels_folder = os.path.join(main, "labels")
images_folder = os.path.join(main, "images")
print("Labels folder exists:", os.path.exists(labels_folder))
print("Images folder exists:", os.path.exists(images_folder))
print("Labels files:", sorted(os.listdir(labels_folder))[:20])  # show first 20 label files


Labels folder exists: True
Images folder exists: True
Labels files: ['labels.csv', 'test_subset0.csv', 'test_subset1.csv', 'test_subset2.csv', 'test_subset3.csv', 'test_subset4.csv', 'train_subset0.csv', 'train_subset1.csv', 'train_subset2.csv', 'train_subset3.csv', 'train_subset4.csv', 'val_subset0.csv', 'val_subset1.csv', 'val_subset2.csv', 'val_subset3.csv', 'val_subset4.csv']


In [ ]:
import pandas as pd

labels_csv = os.path.join(labels_folder, "labels.csv")
print("labels.csv path:", labels_csv)
df_labels = pd.read_csv(labels_csv)
print("columns:", df_labels.columns.tolist())
print("first 6 rows:\n", df_labels.head(6))

# peek one subset at  (test_subset0.csv)
test0 = pd.read_csv(os.path.join(labels_folder, "test_subset0.csv"))
print("\nTest subset example (first 6):\n", test0.head(6))


labels.csv path: /content/drive/MyDrive/data_extracted/labels/labels.csv
columns: ['Filename', 'Label', 'Species']
first 6 rows:
                 Filename  Label       Species
0  20160928-140314-0.jpg      0  Chinee apple
1  20160928-140337-0.jpg      0  Chinee apple
2  20160928-140731-0.jpg      0  Chinee apple
3  20160928-140747-0.jpg      0  Chinee apple
4  20160928-141107-0.jpg      0  Chinee apple
5  20160928-141135-0.jpg      0  Chinee apple

Test subset example (first 6):
                 Filename  Label
0  20160928-140747-0.jpg      0
1  20160928-141437-0.jpg      0
2  20160928-142110-0.jpg      0
3  20161207-110730-0.jpg      0
4  20161207-110753-0.jpg      0
5  20161207-110837-0.jpg      0


In [ ]:
main = "/content/drive/MyDrive/data_extracted"
LABELS_DIR = os.path.join(main, "labels")
IMAGES_DIR = os.path.join(main, "images")

# it disappears bro
print("root exists:", os.path.exists(main))
print("labels dir exists:", os.path.exists(LABELS_DIR))
print("images dir exists:", os.path.exists(IMAGES_DIR))
print("files in labels dir (sample):", sorted(os.listdir(LABELS_DIR))[:20])


root exists: True
labels dir exists: True
images dir exists: True
files in labels dir (sample): ['labels.csv', 'test_subset0.csv', 'test_subset1.csv', 'test_subset2.csv', 'test_subset3.csv', 'test_subset4.csv', 'train_subset0.csv', 'train_subset1.csv', 'train_subset2.csv', 'train_subset3.csv', 'train_subset4.csv', 'val_subset0.csv', 'val_subset1.csv', 'val_subset2.csv', 'val_subset3.csv', 'val_subset4.csv']


In [ ]:
master_csv = os.path.join(LABELS_DIR, "labels.csv")
print("Master CSV:", master_csv)
master_df = pd.read_csv(master_csv)   # contains Filename,Label,Species (as you showed)
print("master_df columns:", master_df.columns.tolist())
print(master_df.head(6))

subset0 = pd.read_csv(os.path.join(LABELS_DIR, "test_subset0.csv"))
print("\nSubset sample:", subset0.head(6))


Master CSV: /content/drive/MyDrive/data_extracted/labels/labels.csv
master_df columns: ['Filename', 'Label', 'Species']
                Filename  Label       Species
0  20160928-140314-0.jpg      0  Chinee apple
1  20160928-140337-0.jpg      0  Chinee apple
2  20160928-140731-0.jpg      0  Chinee apple
3  20160928-140747-0.jpg      0  Chinee apple
4  20160928-141107-0.jpg      0  Chinee apple
5  20160928-141135-0.jpg      0  Chinee apple

Subset sample:                 Filename  Label
0  20160928-140747-0.jpg      0
1  20160928-141437-0.jpg      0
2  20160928-142110-0.jpg      0
3  20161207-110730-0.jpg      0
4  20161207-110753-0.jpg      0
5  20161207-110837-0.jpg      0


In [ ]:
import glob
def concat_subsets(labels_dir, pattern):
    """Concatenate CSV files matching pattern in labels_dir into one DataFrame."""
    paths = sorted(glob.glob(os.path.join(labels_dir, pattern)))
    if not paths:
        raise FileNotFoundError(f"No files for pattern {pattern} in {labels_dir}")
    dfs = [pd.read_csv(p) for p in paths]
    return pd.concat(dfs, ignore_index=True)

train_df = concat_subsets(LABELS_DIR, "train_subset*.csv")
val_df   = concat_subsets(LABELS_DIR, "val_subset*.csv")
test_df  = concat_subsets(LABELS_DIR, "test_subset*.csv")

print("train/val/test sizes:", len(train_df), len(val_df), len(test_df))
print("train sample:\n", train_df.head())


train/val/test sizes: 52525 17511 17509
train sample:
                 Filename  Label
0  20171109-175921-2.jpg      5
1  20170714-142019-3.jpg      1
2  20170718-101402-2.jpg      0
3  20170126-095456-0.jpg      1
4  20170913-110647-1.jpg      3


In [54]:
device = torch.device("cuda")
print("Using device:", device)


Using device: cuda


In [51]:
IMG_SIZE = 224

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [56]:
def load_sample(row, image_dir, transform):
    """
    row: one row from train_df / val_df / test_df
    returns: image tensor, label tensor
    """
    img_path = os.path.join(image_dir, row["Filename"])
    image = Image.open(img_path).convert("RGB")
    image = transform(image)
    label = torch.tensor(int(row["Label"]), dtype=torch.long)
    return image, label


def get_batch(df, image_dir, transform, batch_size, start_idx):
    images = []
    labels = []

    end_idx = min(start_idx + batch_size, len(df))
    for i in range(start_idx, end_idx):
        img, lab = load_sample(df.iloc[i], image_dir, transform)
        images.append(img)
        labels.append(lab)

    images = torch.stack(images)
    labels = torch.stack(labels)
    return images.to(device), labels.to(device)


In [60]:
import timm
num_classes = master_df["Label"].nunique()

model = timm.create_model("efficientnet_b0", pretrained=True)
model.classifier = nn.Linear(model.classifier.in_features, num_classes)
model = model.to(device)
criterion = nn.CrossEntropyLoss()

def train_epoch(df, transform, optimizer, batch_size=32):
    model.train()
    correct = 0
    total = 0

    for i in range(0, len(df), batch_size):
        images, labels = get_batch(df, IMAGES_DIR, transform, batch_size, i)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        preds = outputs.argmax(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return correct / total


In [61]:
def evaluate(df, transform, batch_size=32):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for i in range(0, len(df), batch_size):
            images, labels = get_batch(df, IMAGES_DIR, transform, batch_size, i)
            outputs = model(images)
            preds = outputs.argmax(1)

            all_preds.append(preds.cpu())
            all_labels.append(labels.cpu())

    all_preds = torch.cat(all_preds).numpy()
    all_labels = torch.cat(all_labels).numpy()
    return all_labels, all_preds


In [62]:
for p in model.parameters():
    p.requires_grad = False

for p in model.classifier.parameters():
    p.requires_grad = True

optimizer = optim.Adam(model.classifier.parameters(), lr=1e-3)

for epoch in range(5):
    acc = train_epoch(train_df, train_transform, optimizer)
    print(f"Partial FT Epoch {epoch+1}: Train Acc = {acc:.4f}")


Partial FT Epoch 1: Train Acc = 0.6812
Partial FT Epoch 2: Train Acc = 0.7261
Partial FT Epoch 3: Train Acc = 0.7362
Partial FT Epoch 4: Train Acc = 0.7447
Partial FT Epoch 5: Train Acc = 0.7474


In [63]:
for p in model.parameters():
    p.requires_grad = True

optimizer = optim.Adam(model.parameters(), lr=5e-5)

for epoch in range(5):
    acc = train_epoch(train_df, train_transform, optimizer)
    print(f"Full FT Epoch {epoch+1}: Train Acc = {acc:.4f}")


Full FT Epoch 1: Train Acc = 0.8484
Full FT Epoch 2: Train Acc = 0.9111
Full FT Epoch 3: Train Acc = 0.9364
Full FT Epoch 4: Train Acc = 0.9487
Full FT Epoch 5: Train Acc = 0.9586
